# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("../WeatherPy/output_data/cities.csv")
cities.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
0,3985710,Cabo San Lucas,20,MX,1596402470,52,22.89,-109.91,91.40,13.87
1,2112309,Katsuura,75,JP,1596402470,94,35.13,140.30,75.20,5.82
2,2171099,Codrington,14,AU,1596402471,87,-38.27,141.97,42.66,10.54
3,2155415,New Norfolk,78,AU,1596402471,82,-42.78,147.06,44.01,1.01
4,1015776,Bredasdorp,0,ZA,1596402471,93,-34.53,20.04,48.20,4.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Set variables for locations and humidity to plot on heat map
locations = cities[['Lat', 'Lon']]
humidity = cities['Humidity'].astype(float)
max_humidity = humidity.max()

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_humidity,
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_temp = cities.loc[(cities['Max Temp'] > 70) & (cities['Max Temp'] < 80)]
ideal_wind = ideal_temp.loc[(ideal_temp['Wind Speed'] < 10)]
ideal_weather = ideal_wind.loc[(ideal_wind['Cloudiness'] ==0)]
ideal_weather.dropna()

# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
56,1524302,Ereymentau,0,KZ,1596402484,50,51.62,73.10,70.68,6.51
88,217834,Bukama,0,CD,1596402492,29,-9.20,25.85,72.72,3.22
207,4035715,Avarua,0,CK,1596402520,64,-21.21,-159.78,71.60,3.36
224,1817446,Baisha,0,CN,1596402524,83,26.52,110.93,75.38,1.10
249,5961417,Gimli,0,CA,1596402531,47,50.63,-96.99,73.00,5.99
304,1513491,Yangi Marg`ilon,0,UZ,1596402547,47,40.43,71.72,75.20,6.93
309,1808106,Hongjiang,0,CN,1596402548,86,27.11,110.00,76.39,1.25
367,1169607,Muzaffarabad,0,PK,1596402563,60,34.37,73.47,75.52,4.63
375,161901,Kaka,0,TM,1596402566,21,37.35,59.61,76.51,0.83
469,172515,Abu Samrah,0,SY,1596402591,60,35.30,37.18,73.71,6.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(ideal_weather, columns=['City', 'Country', 'Lat', 'Lon'])
hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Lat,Lon,Hotel Name
56,Ereymentau,KZ,51.62,73.10,
88,Bukama,CD,-9.20,25.85,
205,Lüderitz,NaN,-26.65,15.16,
207,Avarua,CK,-21.21,-159.78,
224,Baisha,CN,26.52,110.93,


In [7]:
# params dictionary to update each iteration
params = {"radius": 5000,
            "types": "hotel",
            "keyword": "hotels",
            "key": g_key}

# Use the lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lon"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))